# Factorizations and other fun
Author: Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)
(with edits from Jane Herriman)

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Let's start by generating a linear system of the form

`Ax = b`

In [ ]:
A = randn(3,3)

In [ ]:
x = fill(1.0, (3))
b = A*x

### Factorization
The `\` function hides how the problem is actually solved. 

Depending on the dimensions of `A`, different methods are chosen to solve the problem

```
Ax = b
```

An intermediate step in the solution is to calculate a factorization of the matrix `A`. 

Basically, a factorization of `A` is a way of expressing `A` as a product of triangular, unitary and permutation matrices. 

Julia stores these factorizations using a `Factorization` abstract type and several composite subtypes.

A `Factorization` object should therefore be thought of as a representation of the matrix `A`.

#### LU

When `A` is square, a linear system is solved by factorizing the matrix `A` via

```
PA=LU
``` 

where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular. 

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

We can perform an LU factorization on `A` via either `lu(A)` or `lufact(A)`.

The function `lu` returns matrices l and u and permutation vector p.

In [ ]:
l,u,p = lu(A)

Pivoting is on by default, so we can't assume that A == LU.

Let's show this by looking at the norm of `LU - A`:

In [ ]:
norm(l*u - A)

This shows that we want to account for pivoting!

You can think of `A[p,:]` as the syntax for `PA`, or the product of our permutation matrix and matrix `A`

In [ ]:
norm(l*u - A[p,:])

On the other hand, we could turn pivoting off for LU factorizations using the argument `Val{false}` in Julia 0.6 or `Val(false)` in later versions.

In [ ]:
l,u,p = lu(A, Val{false})

When pivoting is off, `LU = A`

In [ ]:
norm(l*u - A)

A second way to perform an LU factorization is with the function `lufact`.

In [ ]:
Alu = lufact(A)

The different parts of the factorization can be extracted by special indexing

In [ ]:
Alu[:P]

In [ ]:
Alu[:L]

In [ ]:
Alu[:U]

We can compute the solution of $Ax=b$ from the factorization object

In [ ]:
# PA = LU
# A = P'LU
# P'LUx = b
# LUx = Pb
# Ux = L\Pb
# x = U\L\Pb
Alu[:U]\(Alu[:L]\(Alu[:P]b))

*More importantly* we can dispatch on the `LU` type and simply solve the system by

In [ ]:
Alu\b

This could be useful if the same left-hand-side is used for several right-hand-sides. The factorization can also be used for calculating the determinant because $\det(A)=\det(PLU)=\det(P)\det(U)=\pm \prod u_{ii}$ because $U$ is triangular and the sign is determined from $\det(P)$.

In [ ]:
det(A)

In [ ]:
det(Alu)

#### QR
When `A` is tall, 

In [ ]:
Atall = randn(3, 2)

Julia computes the least squares solution $\hat{x}$ that minimizes $\|Ax-b\|_2$. 

This can be done by factorizing 

```
A=QR
``` 

where $Q$ is unitary/orthogonal and $R=\left(\begin{smallmatrix}R_0\\0\end{smallmatrix}\right)$ and $R_0$ is upper triangular. 

With the QR factorization the minimum norm can be expressed as

\begin{equation*}
\|Ax-b\|=\|QRx-b\|=\|Q(Rx-Q'b)\|=\|Rx-Q'b\|=\left\|\begin{pmatrix}R_0x-Q_0'b\\Q_1'b\end{pmatrix}\right\|=\|R_0x-Q_0'b\|+\|Q_1'b\|
\end{equation*}

and the problem therefore reduces to solving the square problem $R_0x=Q_0'b$ for $x$.

We can QR factorize `Atall` via

In [ ]:
Aqr = qrfact(Atall)

Another feature of the QR factorizations is the `Q` types for storing the unitary matrices $Q$. They can be extracted from the `QR` types by indexing

In [ ]:
Aqr[:Q]

Similarly, the upper triangular matrix $R$ can be extracted by indexing

In [ ]:
Aqr[:R]

In this case, R is stored as a 2x2 matrix rather than as a 3x2 because the last row of R is filled with 0's.<br><br>


Even though the matrix `Aqr[:Q]` is printed as a $3\times 3$ matrix in the factorization object, in practice it can represent the thin version as well. Hence

In [ ]:
Aqr[:Q]*ones(2)

works, and represents multiplying a $3 x 2$ matrix by a 2-element vector.

Similarly,

In [ ]:
Aqr[:Q]*ones(3)

works, representing the multiplication of a $3x3$ matrix and a 3-element vector.

However, this does not mean that we can multiply `Q` by vectors of arbitrary length.

In [ ]:
Aqr[:Q]*ones(4)

The matrix has a compact internal representation, so indexing is only meaningful if you know how the factorization stores data.

In [ ]:
Aqr[:Q][1]

The QRCompactWY object `\` has a method for the QR and the least squares problem is therefore solved with

In [ ]:
Aqr\b

If we had instead written simply

In [ ]:
Atall\b

rather than QR factorizing the matrix `Atall` first, Julia would have defaulted to QR factorizing *with* pivoting.

Defaulting to using pivoting with a QR factorization allows Julia to handle rank deficient problems. 

We can explicitly choose to use pivoting during the QR factorization (of a singular matrix, for example) with the keyword `Val{true}`.

In [ ]:
v = randn(3)
# Taking the outer product of a vector with itself gives a singular matrix
singmatrix = v * v'

In [ ]:
Aqrp = qrfact(singmatrix,Val{true})

Notice that the type of the resulting Factorization object is different now. 


`\` also has a method for `QRPivoted` and the rank deficient problem is therefore computed

In [ ]:
Aqrp\b

#### Eigendecompositions and the SVD(s)

The results from eigendecompositions and singular value decompositions are also stored in `Factorization` types. This also includes Hessenberg and Schur factorizations.

The eigendecomposition can be computed

In [ ]:
Asym = A + A'
AsymEig = eigfact(Asym)

The values and the vectors can be extracted from the Eigen type by special indexing

In [ ]:
AsymEig[:values]

In [ ]:
AsymEig[:vectors]

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*Asym

Julia also has an `eig` function which returns a tuple with the values and the vectors

In [ ]:
eig(Asym)

We do not recommend this version.

The `svdfact` function computes the singular value decomposition

In [ ]:
Asvd = svdfact(A[:,1:2])

and again `\` has a method for the type enabling least squares by SVD

In [ ]:
Asvd\b

There are special functions for providing values only: `eigvals` and `svdvals`.

### Special matrix Structures
The structure of matrices is very important in linear algebra. This structure can be made explicit in Julia through composite types. Examples are `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`. Specialized methods are written for the special matrix types to take advantage of their structure. Below some examples are shown

In [ ]:
A

Creating a diagonal matrix:

In [ ]:
Diagonal(diag(A))

In [ ]:
Diagonal(A)

Creating a lower triangular matrix:

In [ ]:
LowerTriangular(tril(A))

In [ ]:
LowerTriangular(A)

Creating symmetric matrices:

In [ ]:
Symmetric(Asym)

In [ ]:
SymTridiagonal(diag(Asym),diag(Asym,1))

When it is known that a matrix is e.g. triangular or symmetric Julia might be able to solve a problem faster by converting the matrix to a special matrix. 

For some of the procedures, Julia checks if the input matrix is triangular or symmetric and converts the matrix if such a structure is detected. 

It should be noted that `Symmetric`, `Hermitian` and `Triangular` do not copy the input matrix.

#### Symmetric eigenproblem
Whether or not Julia is able to detect if a matrix is symmetric/Hermitian can have a big influence on how fast an eigenvalue problem is solved. 

In [ ]:
n = 1000;
A = randn(n,n);

Let's use `A` to generate a symmetric matrix, `Asym`

In [ ]:
Asym = A + A';

Now let's create a noisy version of Asym to simulate a symmetric matrix with floating point errors.

In [ ]:
Asym_noisy = copy(Asym); Asym_noisy[1,2] += 5eps();

Can Julia determine that both `Asym` and `Asym_noisy` are symmetric matrices?

In [ ]:
println("Is Asym symmetric? ", issymmetric(Asym))
println("Is Asym_noisy symmetric? ", issymmetric(Asym_noisy))

Now let's look at how the noise in `Asym_noisy` impacts the time to perform an eigendecomposition

In [ ]:
@time eigvals(Asym);

In [ ]:
@time eigvals(Asym_noisy);

Luckily we can provide explicit information about matrix structure to Julia.

In this example, we do so with the `Symmetric` keyword

In [ ]:
@time eigvals(Symmetric(Asym_noisy));

And so we've made our calculations substantially more efficient :)

### A big problem
Using tridiagonal matrices makes it possible to work with potentially very large problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a `Matrix` type.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

### Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does. For a long time Julia has also had support for multiplicaton of general element types. Hence, when multiplying integer matrices, the output is also an integer matrix

In [ ]:
rand(1:10,3,3)*rand(1:10,3,3)

Recently, more generic linear algebra methods has been added and Julia now supports generic `LU` and `QR` factorizations. Generic eigenvalue and SVD methods have been written more recently (some in external packages).

In general, the `LU` factorization can be computed whenever the matrix element type is closed under the operations `+`, `-`, `*` and `\`. Of course the matrix also has to have full rank. The generic `LU` method in Julia applies pivoting and therefore the element type also has to support `<` and `abs`. Hence it is possible to solve systems of equations of e.g. rational numbers which the following examples show.

To work with a rational number, use double forward slashes:

In [ ]:
1//2

#### Example 1: Rational linear system of equations
Julia has rational numbers built in. The following example shows how a linear system of equations can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [ ]:
Ar = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

In [ ]:
x = fill(1, (3))
b = Ar*x

In [ ]:
Ar\b

In [ ]:
lufact(Ar)

#### Example 2: Rational matrix from eigenstructure
The next example shows how rational matrix arithmetic can be used for calculating a matrix given rational eigenvalues and -vectors. I have found this convenient when giving examples of linear dynamic systems.

In [ ]:
λ1,λ2,λ3 = 1//1,1//2,1//4
v1,v2,v3 = [1,0,0],[1,1,0],[1,1,1]
V,Λ = [v1 v2 v3], Diagonal([λ1,λ2,λ3])
A = V*Λ/V

### Exercises

11.1 What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

11.2 Create a diagonal matrix from the eigenvalues of A.

11.3 Perform a Hessenberg factorization on matrix A. Verify that `A = QHQ'`.